In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
from verification.utils import load_jsonl
aime2025 = load_jsonl("../data/aime2025.jsonl")
aime2025_qwen32b = [x for x in aime2025 if x["llm_responses"][0]["model"] == "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"]

In [3]:
def verifier_flops(d_model, d_inter, n_layers, vocab_size, T_in, T_out):
    """
    FLOPs estimate for a decoder-only Transformer with KV cache.
    - d_model: hidden size (d)
    - d_inter: MLP intermediate size (m)
    - n_layers: number of transformer layers (L)
    - vocab_size: vocab size (V); set to 1 for discriminative
    - T_in: input/prefill tokens
    - T_out: output tokens; for discriminative, usually 1
    """
    d, m, L, V = d_model, d_inter, n_layers, vocab_size

    # per-layer, per-token constants (QKV+O + MLP)
    const_per_layer_token = 8*d*d + 4*d*m

    # attention coeff per layer (scales with context length)
    attn_coeff_per_layer = 4*d

    # Prefill
    prefill_ctx_sum = T_in * (T_in + 1) // 2
    flops_prefill = L * (
        const_per_layer_token * T_in +
        attn_coeff_per_layer  * prefill_ctx_sum
    )

    # Decode
    decode_ctx_sum = T_in * T_out + (T_out - 1) * T_out // 2 if T_out > 0 else 0
    flops_decode_layers = L * (
        const_per_layer_token * T_out +
        attn_coeff_per_layer  * decode_ctx_sum
    )

    # LM head (de-embedding)
    flops_lm_head = 2 * d * V * T_out

    return flops_prefill + flops_decode_layers + flops_lm_head

solver

In [4]:
import numpy as np

input_lengths = []
output_lengths = []

for item in aime2025_qwen32b:
    propmt = item["prompt"]
    prompt_length = len(tokenizer.encode(propmt))
    input_lengths.append(prompt_length)

    for response_item in item["llm_responses"]:
        response = response_item["response"]
        output_length = len(tokenizer.encode(response))
        output_lengths.append(output_length)

AVERAGE_SOLVER_INPUT_LENGTH = int(np.mean(input_lengths))
AVERAGE_SOLVER_OUTPUT_LENGTH = int(np.mean(output_lengths))
print(f"{AVERAGE_SOLVER_INPUT_LENGTH=}")
print(f"{AVERAGE_SOLVER_OUTPUT_LENGTH=}")

Token indices sequence length is longer than the specified maximum sequence length for this model (16387 > 16384). Running this sequence through the model will result in indexing errors


AVERAGE_SOLVER_INPUT_LENGTH=133
AVERAGE_SOLVER_OUTPUT_LENGTH=10602


In [5]:
AVERAGE_SOLVER_FLOPS_PER_SAMPLE = verifier_flops(5120, 27648, 64, 152064, AVERAGE_SOLVER_INPUT_LENGTH, AVERAGE_SOLVER_OUTPUT_LENGTH)
AVERAGE_SOLVER_FLOPS_PER_SAMPLE

625131826708480

In [6]:
# from scripts/calc_latency.py
AVERAGE_SOLVER_LATENCY_PER_SAMPLE_N_1 = 273.074387550354
AVERAGE_SOLVER_LATENCY_PER_SAMPLE_N_2 = 276.6106116771698
AVERAGE_SOLVER_LATENCY_PER_SAMPLE_N_4 = 288.38619780540466
AVERAGE_SOLVER_LATENCY_PER_SAMPLE_N_8 = 448.35090923309326
AVERAGE_SOLVER_LATENCY_PER_SAMPLE_N_16 = 782.906281709671
AVERAGE_SOLVER_LATENCY_PER_SAMPLE_N_32 = 1439.957554101944
AVERAGE_SOLVER_LATENCY_PER_SAMPLE_N_64 = 2815.468241930008
AVERAGE_SOLVER_LATENCY_PER_SAMPLE_N_128 = 5514.078681468964

heimdall

In [8]:
input_lengths = []

heimdall_prompt = """
Here is a math problem and a solution of it. Think step by step and verify if the final answer in the
solution is correct. The last line of your response should be of the form Answer: $Answer (without
quotes) where $Answer is 1 if the final answer in the solution is correct and 0 if incorrect.

**Problem**
${problem}

**Solution**
${solution}
""".strip()

for item in aime2025_qwen32b:
    problem = item["prompt"]
    for response_item in item["llm_responses"]:
        solution = response_item["response"]
        solution = solution.split("</think>")[-1]

    prompt = heimdall_prompt.format(problem=problem, solution=solution)
    input_length = len(tokenizer.encode(prompt))
    input_lengths.append(input_length)

AVERAGE_HEIMDALL_INPUT_LENGTH = int(np.mean(input_lengths))
print(f"{AVERAGE_HEIMDALL_INPUT_LENGTH=}")

AVERAGE_HEIMDALL_OUTPUT_LENGTH = 10500 # extrapolated from paper's results on aime 2024
print(f"{AVERAGE_HEIMDALL_OUTPUT_LENGTH=}")

AVERAGE_HEIMDALL_INPUT_LENGTH=5525
AVERAGE_HEIMDALL_OUTPUT_LENGTH=10500


In [9]:
AVERAGE_HEIMDALL_FLOPS_PER_SAMPLE = verifier_flops(5120, 27648, 64, 152064, AVERAGE_HEIMDALL_INPUT_LENGTH, AVERAGE_HEIMDALL_OUTPUT_LENGTH)
AVERAGE_HEIMDALL_FLOPS_PER_SAMPLE

980453982208000

In [10]:
# from scripts/calc_latency.py
AVERAGE_HEIMDALL_LATENCY_PER_SAMPLE_N_1 = 276.01961755752563
AVERAGE_HEIMDALL_LATENCY_PER_SAMPLE_N_2 = 279.4113857746124
AVERAGE_HEIMDALL_LATENCY_PER_SAMPLE_N_4 = 328.29437160491943
AVERAGE_HEIMDALL_LATENCY_PER_SAMPLE_N_8 = 496.405907869339
AVERAGE_HEIMDALL_LATENCY_PER_SAMPLE_N_16 = 912.8740525245667
AVERAGE_HEIMDALL_LATENCY_PER_SAMPLE_N_32 = 1711.8473834991455
AVERAGE_HEIMDALL_LATENCY_PER_SAMPLE_N_64 = 3334.3926618099213
AVERAGE_HEIMDALL_LATENCY_PER_SAMPLE_N_128 = 6580.33390879631

our dv

In [11]:
input_lengths = []

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

for item in aime2025_qwen32b:
    prompt = item["prompt"]
    for response_item in item["llm_responses"]:
        response = response_item["response"]

        inputs = tokenizer.apply_chat_template(
            [
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": response.split("</think>")[-1]}
            ],
            tokenize=True, 
            continue_final_message=False,
        )

        input_lengths.append(len(inputs))

AVERAGE_DV_INPUT_LENGTH = int(np.mean(input_lengths))
print(f"{AVERAGE_DV_INPUT_LENGTH=}")

AVERAGE_DV_OUTPUT_LENGTH = 1 # single forward pass
print(f"{AVERAGE_DV_OUTPUT_LENGTH=}")

Token indices sequence length is longer than the specified maximum sequence length for this model (16546 > 16384). Running this sequence through the model will result in indexing errors


AVERAGE_DV_INPUT_LENGTH=5082
AVERAGE_DV_OUTPUT_LENGTH=1


In [12]:
AVERAGE_DV_FLOPS_PER_SAMPLE = verifier_flops(1536, 8960, 28, 1, AVERAGE_DV_INPUT_LENGTH, AVERAGE_DV_OUTPUT_LENGTH) # vocab size is 1 since scalar head
AVERAGE_DV_FLOPS_PER_SAMPLE

12744068803584

In [13]:
AVERAGE_DV_LATENCY_PER_SAMPLE = 0.05195287466049194 # from scripts/calc_latency.py